<a href="https://colab.research.google.com/github/FiddiCoder/Movie-Recommender/blob/main/ml_pipeline/Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
#@title Import Librerie
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os
import tarfile
from six.moves import urllib
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn import datasets, linear_model,tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from mlxtend.evaluate import bias_variance_decomp
from sklearn.metrics import zero_one_loss
from imblearn.over_sampling import SMOTE
from pandas.core.arrays.arrow import dtype

In [124]:
#@title Download e formattazione dataset film

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/Movie-Recommender/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/movies2"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "movies2")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

with open('./dataset/movies2', encoding = "ISO-8859-1") as content:
    colonne = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'imdb_url']
    generi = ['unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy',
          'film-noir',  'horror', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western']
    colonneG = colonne + generi
    movies = pd.DataFrame(columns=colonneG)
    i = 0
    for x in content:
        x = x.split("|")
        x[-1] = x[-1][:-1]
        if x[1][-1] == ' ':
            x[1] = x[1][:-1]
        movies.loc[i] = [word if word!='' else "empty" for word in x]
        i = i + 1
movies['movie_id'] = movies['movie_id'].astype('int64')
movies[generi] = movies[generi].astype('category')



In [125]:
#@title Data cleaning: Tabelle con feature inutili
movies = movies.drop('video_release_date', axis=1)
movies = movies.drop('release_date', axis=1)
movies = movies.drop('imdb_url', axis=1)


In [126]:
#@title Download e formattazione dataset utenti
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/Movie-Recommender/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/users.user"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "users.user")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

colonne_user = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users = pd.read_csv('./dataset/users.user', sep='|', names=colonne_user)


In [127]:
#@title Data cleaning, trasformazione variabili da string a boolean

#Drop feature inutile
users = users.drop('zip_code', axis=1)

#Data cleaning = Eliminazione righe vuote
users = users[users['gender'] != 'empty']
users = users[users['occupation'] != 'empty']

#Colonne occupation e gender trasformate in più colonne coi rispettivi nomi delle variabili per dare loro valori booleani
users['occupation'] = users['occupation'].astype('category')
users['gender'] = users['gender'].astype('category')
one_hot_occ = users.occupation.str.get_dummies()
users = users.drop('occupation',axis=1)
users = users.join(one_hot_occ,how='inner')
one_hot_g = users.gender.str.get_dummies()
users = users.drop('gender',axis=1)
users = users.join(one_hot_g,how='inner')
#users.head(5)

In [128]:
#@title Download dataset chiave per joinare utenti e film
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/Movie-Recommender/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/usersmovies.data"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "usersmovies.data")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

colonnekey = ['user_id', 'movie_id', 'rating', 'timestamp']
key = pd.read_csv('./dataset/usersmovies.data', sep='\t', names=colonnekey)

#Drop feature inutile
key = key.drop('timestamp', axis=1)
#key.head(50)

In [129]:
#@title Join dataset utenti e film
final_dataset = key.merge(movies, left_on='movie_id', right_on='movie_id', how='inner')
final_dataset = final_dataset.merge(users, left_on='user_id', right_on='user_id', how='inner')

#Spostiamo la colonna target in ultima posizione
sposta_colonna = final_dataset.pop("rating")
final_dataset.insert(46, "rating", sposta_colonna)

#final_dataset.head(10)

In [130]:
#@title Trasformazione colonna target in booleana

#Vecchia implementazione, modificata dopo Quarto Test

final_dataset['rating'] = final_dataset['rating'].replace(0,0)
final_dataset['rating'] = final_dataset['rating'].replace(1,0)
final_dataset['rating'] = final_dataset['rating'].replace(2,0)
final_dataset['rating'] = final_dataset['rating'].replace(3,1)
final_dataset['rating'] = final_dataset['rating'].replace(4,1)
final_dataset['rating'] = final_dataset['rating'].replace(5,1)
#final_dataset.head(10)


In [131]:
#@title Feature Selection

#Trasformazione variabili categoriche affinchè compaiano nella matrice di correlazione
le = LabelEncoder()
final_dataset['unknown'] = le.fit_transform(final_dataset['unknown'])
final_dataset['action'] = le.fit_transform(final_dataset['action'])
final_dataset['adventure'] = le.fit_transform(final_dataset['adventure'])
final_dataset['animation'] = le.fit_transform(final_dataset['animation'])
final_dataset['children'] = le.fit_transform(final_dataset['children'])
final_dataset['comedy'] = le.fit_transform(final_dataset['comedy'])
final_dataset['crime'] = le.fit_transform(final_dataset['crime'])
final_dataset['documentary'] = le.fit_transform(final_dataset['documentary'])
final_dataset['drama'] = le.fit_transform(final_dataset['drama'])
final_dataset['fantasy'] = le.fit_transform(final_dataset['fantasy'])
final_dataset['film-noir'] = le.fit_transform(final_dataset['film-noir'])
final_dataset['horror'] = le.fit_transform(final_dataset['horror'])
final_dataset['musical'] = le.fit_transform(final_dataset['musical'])
final_dataset['mystery'] = le.fit_transform(final_dataset['mystery'])
final_dataset['romance'] = le.fit_transform(final_dataset['romance'])
final_dataset['sci-fi'] = le.fit_transform(final_dataset['sci-fi'])
final_dataset['thriller'] = le.fit_transform(final_dataset['thriller'])
final_dataset['war'] = le.fit_transform(final_dataset['war'])
final_dataset['western'] = le.fit_transform(final_dataset['western'])

#Matrice correlazione feature
#sns.set(rc = {'figure.figsize':(48,24)})
#sns.heatmap(final_dataset.corr(), annot = True, fmt='.2g',cmap= 'coolwarm')


In [132]:
#@title Ordinamento correlazioni con feature target
# Calcola la correlazione tra le feature e la colonna del rating
correlation_matrix = final_dataset.corr()
rating_correlation = correlation_matrix['rating'].drop('rating').drop('movie_id').drop('user_id')  # Rimuovi la correlazione con se stessa

# Ordina le feature in base alla correlazione con il rating
sorted_features = rating_correlation.abs().sort_values(ascending=False)

# Stampa le feature ordinate per correlazione con il rating
#print(sorted_features)


<ipython-input-132-776015b7d820>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = final_dataset.corr()


In [133]:
#@title Prendiamo le migliori 24 feature

final_dataset = final_dataset.drop('marketing',axis=1)
final_dataset = final_dataset.drop('unknown',axis=1)
final_dataset = final_dataset.drop('musical',axis=1)
final_dataset = final_dataset.drop('sci-fi',axis=1)
final_dataset = final_dataset.drop('salesman',axis=1)
final_dataset = final_dataset.drop('artist',axis=1)
final_dataset = final_dataset.drop('retired',axis=1)
final_dataset = final_dataset.drop('other',axis=1)
final_dataset = final_dataset.drop('adventure',axis=1)
final_dataset = final_dataset.drop('technician',axis=1)
final_dataset = final_dataset.drop('student',axis=1)
final_dataset = final_dataset.drop('homemaker',axis=1)
final_dataset = final_dataset.drop('animation',axis=1)
final_dataset = final_dataset.drop('F',axis=1)
final_dataset = final_dataset.drop('M',axis=1)
final_dataset = final_dataset.drop('lawyer',axis=1)
final_dataset = final_dataset.drop('programmer',axis=1)
final_dataset = final_dataset.drop('documentary',axis=1)
final_dataset = final_dataset.drop('thriller',axis=1)

#Scartiamo le righe che hanno tutti i generi vuoti
final_dataset = final_dataset.loc[~((final_dataset['drama'] == 0) & (final_dataset['comedy'] == 0) & (final_dataset['war'] == 0)
 & (final_dataset['horror'] == 0)  & (final_dataset['film-noir'] == 0) & (final_dataset['romance'] == 0)
  & (final_dataset['children'] == 0) & (final_dataset['fantasy'] == 0) & (final_dataset['action'] == 0)
  &  (final_dataset['mystery'] == 0) & (final_dataset['crime'] == 0) & (final_dataset['western'] == 0))
 ]

#Scartiamo le righe che hanno tutti i dati dell'utente vuoti
final_dataset = final_dataset.loc[~((final_dataset['healthcare'] == 0) & (final_dataset['age'] == 0) & (final_dataset['executive'] == 0)
 & (final_dataset['writer'] == 0)  & (final_dataset['educator'] == 0) & (final_dataset['administrator'] == 0)
  & (final_dataset['scientist'] == 0) & (final_dataset['engineer'] == 0) & (final_dataset['librarian'] == 0)
  & (final_dataset['doctor'] == 0) & (final_dataset['entertainment'] == 0) & (final_dataset['none'] == 0))]


#final_dataset.head(100)


In [134]:

#@title Decision Tree Oversampling

X = final_dataset.iloc[:,3:27]
y = final_dataset.iloc[:,27:]
x_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

smote = SMOTE()

x_train_oversampled, y_train_oversampled = smote.fit_resample(x_train, y_train)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(x_train_oversampled.values, y_train_oversampled)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    id = final_dataset.iloc[i,0]
    if(movie == 'Birdcage, The (1996)'):
      if(id== 196):
        movie_prova = final_dataset.iloc[i,3:27]
        movie_prova_realvalue = final_dataset.iloc[i, 27]

prediction = dtree.predict([movie_prova])

if(prediction[0] == 0):
  print("Il film Birdcage e l' utente con id 196 non sono compatibili")
  if(movie_prova_realvalue == 0):
    print("Valore reale uguale al valore predetto")
  else:
    print("Valore reale differente dal valore predetto")
else:
  print("Il film Birdcage e l' utente con id 196 sono compatibili")
  if(movie_prova_realvalue == 1):
    print("Valore reale uguale al valore predetto")
  else:
    print("Valore reale differente dal valore predetto")
print("\n")
print("\n")

print("Risultati modello:")
print("\n")


score = cross_val_score(modello, x_train_oversampled, y_train_oversampled, cv = 10)
print("Cross validation: \n",score)
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)

print('-------------------------------')
print('\n')

dtree = tree.DecisionTreeClassifier(random_state=123)
dtree.fit(x_train_oversampled.values,y_train_oversampled.values)
predict = dtree.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree, x_train_oversampled.values, y_train_oversampled.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias) #underfitting altissimo
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,predict))


### After Pruning ###
dtree_prnd = tree.DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)
dtree_prnd.fit(x_train_oversampled.values,y_train_oversampled.values)
predict=dtree_prnd.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree_prnd, x_train_oversampled.values, y_train_oversampled.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss--After pruning: %.3f' % avg_expected_loss)
print('Average bias--After pruning: %.3f' % avg_bias)
print('Average variance--After pruning: %.3f' % avg_var)
print('Sklearn 0-1 loss--After pruning: %.3f' % zero_one_loss(y_test,predict))

Il film Birdcage e l' utente con id 196 non sono compatibili
Valore reale differente dal valore predetto




Risultati modello:


Cross validation: 
 [0.63665645 0.64278419 0.69408437 0.69754105 0.69751728 0.69421747
 0.69052483 0.69264613 0.69885292 0.69665305]


Accuracy modello: 0.5956
Confusion matrix

 [[1999 1398]
 [6383 9459]]
-------------------------------




/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Average expected loss: 0.460
Average bias: 8764.023
Average variance: 0.087
Sklearn 0-1 loss: 0.405
Average expected loss--After pruning: 0.522
Average bias--After pruning: 10022.819
Average variance--After pruning: 0.001
Sklearn 0-1 loss--After pruning: 0.480


In [135]:
#@title Decision Tree Undersampling


# Determina il numero di righe da eliminare casualmente
num_rows_to_drop = 63000  # Specifica il numero di righe da eliminare

# Seleziona solo le righe con rating = 1 e determina l'indice di queste righe
rows_to_drop = final_dataset.loc[final_dataset['rating'] == 1].index

# Genera un array casuale di indici di riga da eliminare
random_rows = np.random.choice(rows_to_drop, size=num_rows_to_drop, replace=False)

# Elimina le righe casuali dal DataFrame
final_dataset.drop(random_rows, inplace=True)


# Conta il numero di righe in cui il valore della colonna è 1
count_1 = len(final_dataset[final_dataset['rating'] == 1])

# Conta il numero di righe in cui il valore della colonna è 0
count_0 = len(final_dataset[final_dataset['rating'] == 0])

# Stampa i risultati
print("Numero di righe con valore 1:", count_1)
print("Numero di righe con valore 0:", count_0)

X = final_dataset.iloc[:,3:27]
y = final_dataset.iloc[:,27:]
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    id = final_dataset.iloc[i,0]
    if(movie == 'Birdcage, The (1996)'):
      if(id== 196):
        movie_prova = final_dataset.iloc[i,3:27]
        movie_prova_realvalue = final_dataset.iloc[i, 27]

prediction = dtree.predict([movie_prova])

if(prediction[0] == 0):
  print("Il film Birdcage e l' utente con id 196 non sono compatibili")
  if(movie_prova_realvalue == 0):
    print("Valore reale uguale al valore predetto")
  else:
    print("Valore reale differente dal valore predetto")
else:
  print("Il film Birdcage e l' utente con id 196 sono compatibili")
  if(movie_prova_realvalue == 1):
    print("Valore reale uguale al valore predetto")
  else:
    print("Valore reale differente dal valore predetto")

print("\n")
print("\n")
print("Risultati modello:")
print("\n")

score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)

print('-------------------------------')
print('\n')

dtree = tree.DecisionTreeClassifier(random_state=123)
dtree.fit(X_train.values,y_train.values)
predict = dtree.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias) #underfitting altissimo
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,predict))


### After Pruning ###
dtree_prnd = tree.DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)
dtree_prnd.fit(X_train.values,y_train.values)
predict=dtree_prnd.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree_prnd, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss--After pruning: %.3f' % avg_expected_loss)
print('Average bias--After pruning: %.3f' % avg_bias)
print('Average variance--After pruning: %.3f' % avg_var)
print('Sklearn 0-1 loss--After pruning: %.3f' % zero_one_loss(y_test,predict))



Numero di righe con valore 1: 16484
Numero di righe con valore 0: 16709
Il film Birdcage e l' utente con id 196 non sono compatibili
Valore reale differente dal valore predetto




Risultati modello:


Cross validation: 
 [0.60429217 0.58810241 0.58509036 0.58245482 0.60301318 0.59510358
 0.61092279 0.57777778 0.59849341 0.59585687]


Accuracy modello: 0.5897
Confusion matrix

 [[2170 1238]
 [1486 1745]]
-------------------------------




/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Average expected loss: 0.499
Average bias: 3310.609
Average variance: 0.178
Sklearn 0-1 loss: 0.410
Average expected loss--After pruning: 0.498
Average bias--After pruning: 3297.278
Average variance--After pruning: 0.056
Sklearn 0-1 loss--After pruning: 0.427
